[Lumibot](https://lumibot.lumiwealth.com) offers a [Pandas backtester](https://lumibot.lumiwealth.com/backtesting.pandas.html) that uses a dictionary of DataFrames, each representing an asset. The following code shows how you can trivially build this from Mallard's DuckDB database.

Lumibot requires the following columns in the DataFrame:

Index:
name: datetime
type: datetime64

Columns:
names: ['open', 'high', 'low', 'close', 'volume']
types: float

In [10]:
import duckdb
import configparser
import os

from lumibot.entities import Asset, Data

# Get config file
config_file = os.getenv('MALLARD_CONFIG')
config = configparser.ConfigParser()
config.read(config_file)

if not config.getboolean('tiingo', 'has_fundamentals_addon'):
    exit(0)

db_file = config['DEFAULT']['db_file']
eod_table = config['tiingo']['eod_table']


def get_pandas_data(symbols: list, is_adjusted=True):
    pandas_data = {}
    for symbol in symbols:
        with duckdb.connect(db_file) as con:
            if is_adjusted:
                df = con.execute(f"""
                SELECT date as datetime, adj_open as open, adj_high as high, adj_low as low, adj_close as close, adj_volume as volume 
                FROM {eod_table} WHERE symbol = '{symbol}'""").df()
            else:
                df = con.execute(f"""
                SELECT date as datetime, open, high, low, close, volume 
                FROM {eod_table} WHERE symbol = '{symbol}'""").df()
        asset = Asset(
            symbol=symbol,
            asset_type=Asset.AssetType.STOCK,
        )
        pandas_data[asset] = Data(
            asset,
            df,
            timestep="day",
        )
    return pandas_data

In [11]:
pandas_data = get_pandas_data(['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL'])
pandas_data[Asset('AAPL')].df.head()

,open,high,low,close,volume
datetime,,,,,
1980-12-12 00:00:00-05:00,0.099442,0.099857,0.099442,0.099442,469034069
1980-12-15 00:00:00-05:00,0.094703,0.094703,0.094253,0.094253,175884975
1980-12-16 00:00:00-05:00,0.087751,0.087751,0.087336,0.087336,105728105
1980-12-17 00:00:00-05:00,0.089480,0.089930,0.089480,0.089480,86441686
1980-12-18 00:00:00-05:00,0.092109,0.092524,0.092109,0.092109,73449673
